# importing liberarys part

In [1]:
import requests # sending requests for html pages

In [2]:
from bs4 import BeautifulSoup # webscraping and handeling html pages

In [3]:
import csv, re, time, random,urllib # other things we need to work with them

# PHASE 1

# Collacting all urls

In [4]:
url1 = "https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html" # it countains 10000 film urls

In [5]:
url2 = "https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html" # it countains 10000 film urls

In [6]:
url3 = "https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html" # it countains 10000 film urls

In [7]:
Three_url_list = [url1, url2, url3] #this list countains three urls

In [8]:
Movies_list = [] # this list will have 30,000 urls

In [9]:
def Generate_all_links(URL):
    """ This function will find and all urls and append them into a list """
    response = requests.get(URL) # geting response from website
    soup = BeautifulSoup(response.text, "html.parser") # webscraping in html form
    for link in soup.find_all("a", href = True): # will find all anchore tages in the page
        Movies_list.append(link["href"]) # will append urls into Move_list

In [10]:
for url in Three_url_list: # send one by one three main urls to function
    Generate_all_links(url)

In [11]:
len(Movies_list)# if you can see 30000 url in list it means you have successfully gathered all urls

30000

# Download HTML files part

In [12]:
# location of downloading 30000 html fiels:

In [4]:
cd E:/Python/Python_and_data/GIT_projects/search_engine/Html_fiels

E:\Python\Python_and_data\GIT_projects\search_engine\All_HTML_fiels


In [5]:
with open("E:/Python/Python_and_data/GIT_projects/search_engine/Html_fiels/number_check.txt","r") as read_num: # because of large size we need to download and time that will take it's better to have memory wich can show us were in wich part of the great download and so we can countineu from that part if anything went wrong so that we dont need to start from Article0.
    CHECK_past = read_num.read() # reading the number from text file

In [ ]:
for movie in range(int(CHECK_past),len(Movies_list)): # here we have mechanism wich will start from number in the text file and looping from where it was intrubted
    
    ## check page status  and delay part##
    URL = Movies_list[movie] # get url form list
    Check_blouck = requests.get(URL) # get response from wikipedia page
    if int(Check_blouck.status_code) == 200: # if page is avalibul have 1 or 2 or 3 secounds delay then start opration 
        time.sleep(random.randint(1,2))
    elif int(Check_blouck.status_code) == 429: # if wikipedia block our ip addres dealy for 20 minuts then start opration
        time.sleep(1200) 
        
    ## opration part ##
    if int(Check_blouck.status_code) == 200: # here we need to check that we dont face with 404 and everithng is okey to start opration
        URL = Movies_list[movie] # get url form list
        Response = urllib.request.urlopen(URL) # get response from page and save it for reading pagr with using urllib library
        Web_countent = Response.read() # reading detaile of page
        Final_file = open("Article_{0}.html".format(movie),"wb") # open a html file for write page details in it
        Final_file.write(Web_countent) # write page details in html file
        Final_file.close() # close file
        with open("E:/Python/Python_and_data/GIT_projects/search_engine/Html_fiels/number_check.txt","w") as write_num: # now that we successfully finished opration we need to send its number to file(it means we have downloaded up to this number from 30000)
            WRITE_NUMBER = write_num.write(str(movie))
    else:
        with open("E:/Python/Python_and_data/GIT_projects/search_engine/Html_fiels/number_check.txt","w") as write_num: # if file has a problem we will pass it and write its number to text file
            WRITE_NUMBER = write_num.write(str(movie))
        pass

# collect data from html files

In [68]:
cd E:\Python\Python_and_data\GIT_projects\search_engine\Html_fiels

E:\Python\Python_and_data\GIT_projects\search_engine\Html_fiels


In [69]:
def html_files_data(Num):
    """This function will collect information from html files"""
    open_HTML_file = open("Article_{0}.html".format(Num),"r",encoding="utf-8") # opening html file
    soup = BeautifulSoup(open_HTML_file,"html.parser") # use soup to read info in html parser form
    
    ## Title part ##
    title = soup.find("h1") # h1 is the biggest fount in html and titles are the only one wich use h1
    Title = title.text # text the title
    
    ## intro and plot ##
    list1 = [] # empty list
    for element in soup.find_all(["h2","p"]): # if we want to find intro and plot part we need to search for h2 and p
        list1.append(element.text) # append those elements wich matched in to empty list
        
    ## intro part ##
    ## intro part in this senario can be find in two position ## 
    if len(list1[0]) == 1: # check for first location 
            Intro = list1[1] # if frist location was empty we will select secound location
    else: 
        Intro = list1[0] # else we will select first location
        if "This article needs additional citations" in Intro:
            Intro = "NA"
        
    ## specify contents ##
    
    ## here we will try to find plot part with content table elements
    try:
        list2 = [] # this lsit will countains contents table elements 
        for i in soup.find_all("span"): # we use span to specify those elements
            list2.append(i.text) # append it to the list
        a = list2.index('1') # wikipedia content table first element  is plot
        Start_plot = list2[a+1] # we will set start point after heading ======> for example (Plot[edit])
        b = list2.index('2') # wikipedia content table secound element  is other element after plot
        Finish_plot = list2[b+1] # we will set finish point after heading ======> for example (summery[edit])
        # we actually need those texts between start point and finish point
        ## plot part ##
        for point in list1: #looping in list1 for plot part
            if point == Start_plot+"[edit]": # we will plus [edit] to Start point
                Start = list1.index(point) # we need to find its position in list
            if point == Finish_plot+"[edit]": #  we will plus [edit] to end point
                Finish = list1.index(point) # we need to find its position in list
        Plot = list1[Start+1:Finish] # plot part is now in Plot list
    except: # some times wiki page doesn't have content table 
        list2 = [] # we will append all h2 headings in list2
        for H2 in soup.find_all("h2"): # find all h2 in html file
            list2.append(H2.text) # apped H2 in list2
        try: # here we will face with two senario: 1-plot part exist 2-plot part dosn't exist
            # 1st senario:
            Start_plot = list2[0] # first element of list2 is start point of plot
            Finish_plot = lis2[1] # seconde element of list2 is finish point of plot
            for point in list1: #looping in list1 for plot part
                if point == Start_plot: # start point is start point of the plot text
                    Start = list1.index(point) # we need to find its position in list
                if point == Finish_plot: # finish point is finish point of the plot text
                    Finish = list1.index(point) # we need to find its position in list
            Plot = list1[Start+1:Finish] # plot text
            # 2nd senario:
        except:
            Plot = "NA" # we will set Plot as NA 
            pass
    ## information table part ##
    # in some wiki pages we dont have information table so we will face with two senario:  1-info table exist 2-info table dosn't exist
    # 1st senario:
    try:
        list3 = [] # this list will have all trs in tbody 
        table = soup.find("table",{"class":"infobox vevent"}) # we will search specifeid table (infobox vevent)
        tbody = table.find("tbody") # find table part in table
        for tr in tbody.find_all("tr"): # find all trs
            list3.append(tr.text) # append all trs to list3
        
        Film_name = list3[0] # first element is movie name
        # by defult we will set all of infos as NA if we find value for them it will change
        Directed_by = "NA"
        Produced_by = "NA"
        Written_by = "NA"
        Starring = "NA"
        Music_by = "NA"
        Release_date = "NA"
        Running_time = "NA"
        Country = "NA"
        Language = "NA"
        Budget = "NA" 
        
        for find_info in range(0,len(list3)): # looping in list3 for trs
            Info_element = list3[find_info] 
            
            if "Directed by" in Info_element: # if Directed by is in Info_element
                Splited_info = Info_element.split("Directed by") # we will use Directed by as key word to split text
                Directed_by = Splited_info[1] # 2nd element of Splited_info is Director information
            
            if "Produced by" in Info_element:# if Produced by is in Info_element
                Splited_info = Info_element.split("Produced by") # we will use Produced by as key word to split text
                Produced_by = Splited_info[1] #2nd element of Splited_info is Producer information
            
            if "Written by" in Info_element:# if Written by is in Info_element
                Splited_info = Info_element.split("Written by")# we will use Written by as key word to split text
                Written_by = Splited_info[1] #2nd element of Splited_info is writer information
            
            if "Starring" in Info_element:# if Starring is in Info_element
                Splited_info = Info_element.split("Starring")# we will use Starring as key word to split text
                Starring = Splited_info[1]#2nd element of Splited_info is Starring information
        
            if "Music by" in Info_element:# if Music by is in Info_element
                Splited_info = Info_element.split("Music by")# we will use Music by as key word to split text
                Music_by = Splited_info[1]#2nd element of Splited_info is Music by information
            
            if "Release date" in Info_element:# if Release date is in Info_element
                Splited_info = Info_element.split("Release date")# we will use Release date as key word to split text
                Release_date = Splited_info[1]#2nd element of Splited_info is Release date information
            
            if "Running time" in Info_element:# if Running time is in Info_element
                Splited_info = Info_element.split("Running time")# we will use Running time as key word to split text
                Running_time = Splited_info[1]#2nd element of Splited_info is Running time information
            
            if "Country" in Info_element:# if Country is in Info_element
                Splited_info = Info_element.split("Country")# we will use Country as key word to split text
                Country = Splited_info[1]#2nd element of Splited_info is Country information
            
            if "Language" in Info_element:# if Language is in Info_element
                Splited_info = Info_element.split("Language")# we will use Language as key word to split text
                Language = Splited_info[1]#2nd element of Splited_info is Language information
        
            if "Budget" in Info_element:# if Budget is in Info_element
                Splited_info = Info_element.split("Budget")# we will use Budget as key word to split text
                Budget = Splited_info[1]#2nd element of Splited_info is Budget information
                
            # write tsv file    
            with open ("E:\Python\Python_and_data\GIT_projects\search_engine\TSV_files\Movie{0}.tsv".format(Num),"w") as tsv_output:
                tsv_writer = csv.writer(tsv_output, delimiter='\t')
                tsv_writer.writerow([Title,Intro,Plot,Film_name,Directed_by,Produced_by,Written_by,Starring,Music_by,Release_date,Running_time,Country,Language,Budget])
    except: # 2nd senario:
        Film_name = "NA"
        Directed_by = "NA"
        Produced_by = "NA"
        Written_by = "NA"
        Starring = "NA"
        Music_by = "NA"
        Release_date = "NA"
        Running_time = "NA"
        Country = "NA"
        Language = "NA"
        Budget = "NA"
        with open ("E:\Python\Python_and_data\GIT_projects\search_engine\TSV_files\Movie{0}.tsv".format(Num),"wt",encoding="utf-8") as tsv_output:
                tsv_writer = csv.writer(tsv_output, delimiter='\t')
                tsv_writer.writerow([Title,Intro,Plot,Film_name,Directed_by,Produced_by,Written_by,Starring,Music_by,Release_date,Running_time,Country,Language,Budget])
        pass

In [70]:
for i in range(0,30000):
    try:
        html_files_data(i)
    except:
        pass

# PHASE 2

In [4]:
cd E:\Python\Python_and_data\GIT_projects\search_engine\TSV_files

E:\Python\Python_and_data\GIT_projects\search_engine\TSV_files


# NLTK part

In [5]:
from nltk.tokenize import word_tokenize , sent_tokenize # for tokenizing words
from nltk.corpus import stopwords # for stop words
from nltk.stem import PorterStemmer # for stemming
import string # for working with strings (in this case for punctuations)

In [7]:
def natural_language_process(Number):
    with open("Movie{0}.tsv".format(Number), "r", encoding="ISO-8859-1") as Read_TSV:
        Data = Read_TSV.read()
        Data = Data.split("\t")
    Intro = Data[1]
    Plot = Data[2]
    
        ## tokenizing ##
    token_intro = word_tokenize(Intro)
    token_plot = word_tokenize(Plot)
    
    ## Remove Stop words ##
    RSW_list_intro = []
    RSW_list_plot = []
    Stop_words = set(stopwords.words("english"))
    #intro part
    for SW in token_intro:
        if SW not in Stop_words:
            RSW_list_intro.append(SW)
    # plot part         
    for SW in token_plot:
        if SW not in Stop_words:
            RSW_list_plot.append(SW)
            
    
    ## Stemming part ##
    Stemm_list_intro = []
    Stemm_list_plot = []
    PS = PorterStemmer()
    # intro part
    for stemming_element in RSW_list_intro:
        Stemm_list_intro.append(PS.stem(stemming_element))
    # plot part
    for stemming_element in RSW_list_plot:
        Stemm_list_plot.append(PS.stem(stemming_element))

    ## Remove punctuation ##
    Remove_punctuation_intro = []
    Remove_punctuation_plot = []
    punctuation_elements = string.punctuation
    # intro part
    for RP in Stemm_list_intro:
        if RP not in punctuation_elements:
            Remove_punctuation_intro.append(RP)
    # plot part
    for RP in Stemm_list_plot:
        if RP not in punctuation_elements:
            Remove_punctuation_plot.append(RP)
    
    ## anything else ##
    
    ## number filter
    Filter_1_intro = []
    Filter_1_plot = []
    nums = ["1","2","3","4","5","6","7","8","9"]
    # intro part
    for AE in Remove_punctuation_intro:
        if AE not in nums:
            Filter_1_intro.append(AE)
    # plot part
    for AE in Remove_punctuation_plot:
        if AE not in nums:
            Filter_1_plot.append(AE)
            
    ## opostrophs filter
    Filter_2_intro = []
    Filter_2_plot = []
    opostroph = ["'S","'s","'m","'re","'t"]
    # intro part
    for s in Filter_1_intro:
        if s not in opostroph:
            Filter_2_intro.append(s)
    # plot part
    for s in Filter_1_plot:
        if s not in opostroph:
            Filter_2_plot.append(s)
            
    ## \\n part        
    joined_intro = " ".join(Filter_2_intro)
    joined_plot = " ".join(Filter_2_plot)
    
    rr = joined_intro.split(".\\n")
    ff = joined_plot.split(".\\n")
    
    ## \\
    joined_intro = " ".join(rr)
    joined_plot = " ".join(ff)
    
    rr = joined_intro.split("\\")
    ff = joined_plot.split("\\")
    
    ## unicodes: \x94 , \x92 , ...
    joined_intro = " ".join(rr)
    joined_plot = " ".join(ff)
    
    # intro part
    printable = set(string.printable)
    intro_ascii = ''.join(filter(lambda x: x in printable, joined_intro))
    # plot part
    printable = set(string.printable)
    plot_ascii = ''.join(filter(lambda x: x in printable, joined_plot))
    
    asembeled_intro = intro_ascii.split(" ")
    asembeled_plot = plot_ascii.split(" ")
    
    # remove elements that didn't exist in string.punctuation
    
    Filter_3_intro = []
    Filter_3_plot = []
    PATERN = ["''","``",'""','',"..."]
    # intro part
    for s in asembeled_intro:
        if s not in PATERN:
            Filter_3_intro.append(s)
    # plot part
    for s in asembeled_plot:
        if s not in PATERN:
            Filter_3_plot.append(s)
    print(Filter_3_intro)
    print(Filter_3_plot)

In [12]:
natural_language_process(12878)

['con', 'air', '1997', 'american', 'action', 'crime', 'film', 'direct', 'simon', 'west', 'written', 'scott', 'rosenberg', 'produc', 'jerri', 'bruckheim', 'the', 'film', 'star', 'nicola', 'cage', 'john', 'cusack', 'john', 'malkovich', 'steve', 'buscemi', 'ving', 'rhame', 'colm', 'meaney', 'mykelti', 'williamson', 'dave', 'chappel', 'rachel', 'ticotin', 'danni', 'trejo', 'nick', 'chinlund', 'jess', 'borrego', 'jose', 'zuniga', 'monica', 'potter', 'support', 'roles.']
["'honor", 'discharg', 'armi', 'ranger', 'sergeant', 'cameron', 'poe', 'return', 'home', 'pregnant', 'wife', 'tricia', 'seri', 'combat', 'deploy', 'howev', 'sentenc', 'ten', 'year', 'prison', 'inadvert', 'kill', 'drunken', 'man', 'one', 'three', 'tri', 'attack', 'tricia', 'while', 'prison', 'poe', 'commun', 'tricia', 'newborn', 'daughter', 'casey', 'letter', 'eight', 'year', 'later', 'parol', 'board', 'flight', 'alabama', 'board', 'jailbird', 'fairchild', 'c-123', 'provid', 'convert', 'fli', 'prison', 'transport', 'poe', 'ac